In [1]:
import torchvision
import torch
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

import timm
from datasets import load_dataset, DatasetDict
from torch.utils.data import DataLoader
from torchvision import transforms

from torch import optim
import sklearn.metrics as metrics
from sklearn.metrics import classification_report

Carrega o dataset e o divide em: treino, teste e validação

In [2]:
dataset = load_dataset("cifar100")

split = dataset["train"].train_test_split(test_size=0.1, seed=42)

dataset = DatasetDict({"train": split["train"], "val": split["test"], "test": dataset["test"]})

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Aplica as transformações e argumentações no datase

In [3]:
transform_224 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [4]:
def transform_batch(example_batch):
    example_batch["pixel_values"] = [transform_224(img.convert("RGB")) for img in example_batch["img"]]
    example_batch["labelsc"] = example_batch["fine_label"]
    example_batch["labelssc"] = example_batch["coarse_label"]
    del example_batch["img"]
    return example_batch

dataset["train"].set_transform(transform_batch)
dataset["val"].set_transform(transform_batch)
dataset["test"].set_transform(transform_batch)

In [5]:
train_loader = DataLoader(dataset["train"], batch_size=128, shuffle=True, num_workers=2)
val_loader = DataLoader(dataset["val"], batch_size=128, shuffle=False, num_workers=2)
test_loader = DataLoader(dataset["test"], batch_size=128, shuffle=False, num_workers=2)

In [6]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

modelo 100 classes

In [7]:
model = timm.create_model('mobilenetv3_small_100', pretrained=True, num_classes=100)
model.to(device)

loss_train = []
loss_eval  = []
patience_time = 15
criterion = nn.CrossEntropyLoss()
opt = optim.AdamW(model.parameters(),lr=0.01)
epochs = 100
epoch = 0

Treino

In [ ]:
stop = False
lowest_loss_eval = 10000
last_best_result = 0

while (not stop):
    model.train()
    lloss = []
    for batch in train_loader:
        images = batch["pixel_values"].to(device)
        labels = batch["labelsc"].to(device)
        images = images.to(device)
        labels = labels.to(device)
        pred = model(images)
        closs = criterion(pred,labels)
        closs.backward()
        opt.step()
        opt.zero_grad()
        lloss.append(closs.item())
    loss_train.append(np.mean(lloss))
    lloss = []
    model.eval()
    lres = []
    lbtrue = []
    with torch.no_grad():
        for batch in val_loader:
            images = batch["pixel_values"].to(device)
            labels = batch["labelsc"].to(device)
            images = images.to(device)

            pred = model(images)
            closs = criterion(pred.cpu(),labels)
            lloss.append(closs.item())
            res  = pred.argmax(dim=1).cpu().tolist()
            lres += res
            lbtrue += labels
        avg_loss_eval = np.mean(lloss)
        loss_eval.append(avg_loss_eval)
        if avg_loss_eval < lowest_loss_eval:
            lowest_loss_eval = avg_loss_eval
            last_best_result = 0
            print("Best model found! saving...")
            actual_state = {'optim':opt.state_dict(),'model':model.state_dict(),'epoch':epoch,'loss_train':loss_train,'loss_eval':loss_eval}
            torch.save(actual_state,'best_modelc.pth')
        last_best_result += 1
        if last_best_result > patience_time:
            stop = True
        print("epoch %d loss_train %4.3f loss_eval %4.3f last_best %d"%(epoch,loss_train[-1],loss_eval[-1],last_best_result))
        epoch += 1

classification report

In [ ]:
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        images = batch["pixel_values"].to(device)
        labels = batch["labelsc"].to(device)

        outputs = model(images)
        preds = outputs.argmax(dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())


print(classification_report(all_labels, all_preds))

modelo 20 super-classes

In [ ]:
model = timm.create_model('mobilenetv3_small_100', pretrained=True, num_classes=20)
model.to(device)

loss_train = []
loss_eval  = []
patience_time = 15
criterion = nn.CrossEntropyLoss()
opt = optim.AdamW(model.parameters(),lr=0.01)
epochs = 100
epoch = 0

Treino

In [ ]:
stop = False
lowest_loss_eval = 10000
last_best_result = 0

while (not stop):
    model.train()
    lloss = []
    for batch in train_loader:
        images = batch["pixel_values"].to(device)
        labels = batch["labelssc"].to(device)
        images = images.to(device)
        labels = labels.to(device)
        pred = model(images)
        closs = criterion(pred,labels)
        closs.backward()
        opt.step()
        opt.zero_grad()
        lloss.append(closs.item())
    loss_train.append(np.mean(lloss))
    lloss = []
    model.eval()
    lres = []
    lbtrue = []
    with torch.no_grad():
        for batch in val_loader:
            images = batch["pixel_values"].to(device)
            labels = batch["labelssc"].to(device)
            images = images.to(device)

            pred = model(images)
            closs = criterion(pred.cpu(),labels)
            lloss.append(closs.item())
            res  = pred.argmax(dim=1).cpu().tolist()
            lres += res
            lbtrue += labels
        avg_loss_eval = np.mean(lloss)
        loss_eval.append(avg_loss_eval)
        if avg_loss_eval < lowest_loss_eval:
            lowest_loss_eval = avg_loss_eval
            last_best_result = 0
            print("Best model found! saving...")
            actual_state = {'optim':opt.state_dict(),'model':model.state_dict(),'epoch':epoch,'loss_train':loss_train,'loss_eval':loss_eval}
            torch.save(actual_state,'best_modelsc.pth')
        last_best_result += 1
        if last_best_result > patience_time:
            stop = True
        print("epoch %d loss_train %4.3f loss_eval %4.3f last_best %d"%(epoch,loss_train[-1],loss_eval[-1],last_best_result))
        epoch += 1

classification report

In [ ]:
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        images = batch["pixel_values"].to(device)
        labels = batch["labelssc"].to(device)

        outputs = model(images)
        preds = outputs.argmax(dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())


print(classification_report(all_labels, all_preds))

modelo multihead

In [27]:
class MultiHeadMobileNetV3(nn.Module):
    def __init__(self, pretrained=True, num_classes_fine=100, num_classes_coarse=10):
        super().__init__()

        base_model = timm.create_model("mobilenetv3_small_100", pretrained=True, num_classes=0)

        self.backbone = base_model


        in_features = 1024
        self.head_fine = nn.Linear(in_features, num_classes_fine)
        self.head_coarse = nn.Linear(in_features, num_classes_coarse)

    def forward(self, x):
        feats = self.backbone(x)
        out_fine = self.head_fine(feats)
        out_coarse = self.head_coarse(feats)
        return out_fine, out_coarse

In [ ]:
model = MultiHeadMobileNetV3()
model.to(device)

loss_train = []
loss_eval  = []
patience_time = 15
criterion = nn.CrossEntropyLoss()
opt = optim.AdamW(model.parameters(),lr=0.01)
epochs = 100
epoch = 0

In [ ]:
Treino

In [ ]:
stop = False
lowest_loss_eval = 10000
last_best_result = 0

while (not stop):
    model.train()
    lloss = []
    for batch in train_loader:
        images = batch["pixel_values"].to(device)
        labels_fine = batch["labelsc"].to(device)
        labels_coarse = batch["labelssc"].to(device)
        images = images.to(device)
        labelsc = labels.to(device)
        pred_fine, pred_coarse = model(images)
        loss_fine = criterion(pred_fine, labels_fine)
        loss_coarse = criterion(pred_coarse, labels_coarse)
        loss = 0.7 * loss_fine + 0.3 * loss_coarse
        loss.backward()
        opt.step()
        opt.zero_grad()
        lloss.append(loss.item())
    loss_train.append(np.mean(lloss))
    lloss = []
    model.eval()
    lres = []
    lbtrue = []
    with torch.no_grad():
        for batch in val_loader:
            images = batch["pixel_values"].to(device)
            labels_fine = batch["labelsc"].to(device)
            labels_coarse = batch["labelssc"].to(device)
            images = images.to(device)

            pred_fine, pred_coarse = model(images)
            loss_fine = criterion(pred_fine.cpu(), labels_fine)
            loss_coarse = criterion(pred_coarse.cpu(), labels_coarse)
            loss = 0.7 * loss_fine + 0.3 * loss_coarse
            lloss.append(loss.item())
            res  = pred.argmax(dim=1).cpu().tolist()
            lres += res
            lbtrue += labels
        avg_loss_eval = np.mean(lloss)
        loss_eval.append(avg_loss_eval)
        if avg_loss_eval < lowest_loss_eval:
            lowest_loss_eval = avg_loss_eval
            last_best_result = 0
            print("Best model found! saving...")
            actual_state = {'optim':opt.state_dict(),'model':model.state_dict(),'epoch':epoch,'loss_train':loss_train,'loss_eval':loss_eval}
            torch.save(actual_state,'best_modelsc.pth')
        last_best_result += 1
        if last_best_result > patience_time:
            stop = True
        print("epoch %d loss_train %4.3f loss_eval %4.3f last_best %d"%(epoch,loss_train[-1],loss_eval[-1],last_best_result))
        epoch += 1

classification report

In [ ]:
all_preds_fine = []
all_labels_fine = []
all_preds_coarse = []
all_labels_coarse = []
with torch.no_grad():
    for batch in test_loader:
        images = batch["pixel_values"].to(device)
        labels_fine = batch["labelsc"].to(device)
        labels_coarse = batch["labelssc"].to(device)

        out_fine, out_coarse = model(images)

        preds_fine = out_fine.argmax(dim=1).cpu().numpy()
        preds_coarse = out_coarse.argmax(dim=1).cpu().numpy()

        all_preds_fine.extend(preds_fine)
        all_labels_fine.extend(labels_fine.cpu().numpy())
        all_preds_coarse.extend(preds_coarse)
        all_labels_coarse.extend(labels_coarse.cpu().numpy())


print(classification_report(all_labels, all_preds))